# Credit card validation using DFAS

Conditions:
1. All American Express numbers start with 34 or 37, 15-digit numbers
2. Most MasterCard numbers start with 51, 52, 53, 54, or 55 (they also have some other potential starting numbers which we won’t concern ourselves with for this problem)- 16 digit numbers
3. Visa numbers start with 4. 13- and 16-digit numbers. 
4. Fulfill Luhn’s Algorithm

- Multiply every other digit by 2, starting with the number’s second-to-last digit, and then add those products’ digits together.
- Add the sum to the sum of the digits that weren’t multiplied by 2.
- If the total’s last digit is 0 (or, put more formally, if the total modulo 10 is congruent to 0), the number is valid!


In [1]:
CREDIT_CARD_NUMBER = 4111111111111111

# Standard algorithm 


In [2]:
def validate_credit_card(number):
    # Cast to string, dealing with digits only
    number = str(number)
    # count digits
    n = len(number)
    
    # Check valid lengths
    if n not in [13, 15, 16]:
        return "INVALID"
    
    # Luhn's algorithm: Process from right to left
    total_sum = 0
    for i in range(n):
        digit = int(number[n - 1 - i])
        if i % 2 == 1:  # Starting from Second-to-last digit and every other digit
            doubled = digit * 2
            # If doubled value is greater than 9(two digit number), sum the digits, not the number
            total_sum += (doubled // 10) + (doubled % 10) if doubled >= 10 else doubled
        else:
            # Add the digits that are not doubled
            total_sum += digit

    #check for Lunh's algorithm validity
    if total_sum % 10 != 0:
        return "INVALID"
    
    # If we havent returned invalid, means it passed Lunhs algorithm, so Determine card type
    first_digit = number[0]
    first_two = number[:2]
    
    if first_two in ["34", "37"] and n == 15:
        return "AMEX"
    elif first_two in ["51", "52", "53", "54", "55"] and n == 16:
        return "MASTERCARD"
    elif first_digit == "4" and n in [13, 16]:
        return "VISA"
    else:
        return "INVALID"

# Test the standard validator
result = validate_credit_card(CREDIT_CARD_NUMBER)
print(f"Standard Validation Result: {result}")

Standard Validation Result: VISA


# DFA algorithm
(A python implementation of a DFA)

In [3]:
#DFA implementation of the credit card validation algorithm
# Three helper functions: 1.update the Luhn sum(as we read digits), 2.state transitions, and determine final card type.
def update_sum(current_sum, digit, k, n):
    """
    When called, Updates the Luhn sum for the DIGIT at position k (from left) for a total length n.
    Doubles the digit if its position from the right (n - k + 1) is even.
    """
    #cast to int
    digit = int(digit)
    # Multiply every other digit by 2, starting with the number’s second-to-last digit
    # Double if (n - k + 1) is even: for odd n, k even; for even n, k odd
    if (n % 2 == 0 and k % 2 == 1) or (n % 2 == 1 and k % 2 == 0):
        doubled = digit * 2
        add = (doubled // 10) + (doubled % 10)
    else:
        add = digit
    return (current_sum + add) % 10

def next_state(state, digit):
    """
    Computes the next DFA state based on the current state and input digit.
    """
    # Handle invalid state
    if state == "invalid":
        return "invalid"
    
    # unpack the current state(type of current state, digit position in original number, and Luhn sum)
    # a pointer keeps the rest of the information
    type_, *rest = state
    # Based on the first digit, the DFA transitions to a "potential" state 
    if type_ == "start":
        if digit == "3":
            return ("potential_Amex", 1, update_sum(0, digit, 1, 15))
        elif digit == "4":
            sum13 = update_sum(0, digit, 1, 13)
            sum16 = update_sum(0, digit, 1, 16)
            return ("Visa", 1, sum13, sum16)
        elif digit == "5":
            return ("potential_MasterCard", 1, update_sum(0, digit, 1, 16))
        else:
            return "invalid"
    
    # Transitions for potential states
    # if the second digit is 4 or 7 for potential Amex go to "Amex" state
    elif type_ == "potential_Amex":
        pos, sum_mod10 = rest
        if pos == 1 and digit in ["4", "7"]:
            return ("Amex", 2, update_sum(sum_mod10, digit, 2, 15))
        return "invalid"
    # if the second digit is 1, 2, 3, 4, or 5 for potential MasterCard go to "MasterCard" state
    elif type_ == "potential_MasterCard":
        pos, sum_mod10 = rest
        if pos == 1 and digit in ["1", "2", "3", "4", "5"]:
            return ("MasterCard", 2, update_sum(sum_mod10, digit, 2, 16))
        return "invalid"
    
    # Transitions for valid states
    # if the type is Amex, MasterCard, or Visa, we continue processing digits
    elif type_ == "Amex":
        pos, sum_mod10 = rest
        if pos < 15:
            return ("Amex", pos + 1, update_sum(sum_mod10, digit, pos + 1, 15))
        return "invalid"
    
    elif type_ == "MasterCard":
        pos, sum_mod10 = rest
        if pos < 16:
            return ("MasterCard", pos + 1, update_sum(sum_mod10, digit, pos + 1, 16))
        return "invalid"
    
    elif type_ == "Visa":
        pos, sum13, sum16 = rest
        if pos < 16:
            new_sum13 = update_sum(sum13, digit, pos + 1, 13)
            new_sum16 = update_sum(sum16, digit, pos + 1, 16)
            return ("Visa", pos + 1, new_sum13, new_sum16)
        return "invalid"
    
    return "invalid"

def get_card_type(state):
    """
    Determines the card type based on the final DFA state.
    """
    if state == "invalid":
        return "INVALID"
    # unpack the final state(type of current state, digit position in original number, and Luhn sum)
    type_, *rest = state
    # check the type, final digitp postion(length), and Luhn sum %10 =0
    if type_ == "Amex" and rest[0] == 15 and rest[1] == 0:
        return "AMEX"
    elif type_ == "MasterCard" and rest[0] == 16 and rest[1] == 0:
        return "MASTERCARD"
    elif type_ == "Visa":
        pos, sum13, sum16 = rest
        if pos == 13 and sum13 == 0:
            return "VISA"
        elif pos == 16 and sum16 == 0:
            return "VISA"
    return "INVALID"

def dfa_validate(number):
    """
    Validates a credit card number using a DFA.
    """
    # initialize the DFA state
    state = ("start",)
    for digit in str(number):
        state = next_state(state, digit)
    return get_card_type(state)

# Test the DFA validator
result = dfa_validate(CREDIT_CARD_NUMBER)
print(f"DFA Validation Result: {result}")

DFA Validation Result: VISA


# Standard Test Cards

Taken from Pay Pal
https://developer.paypal.com/api/nvp-soap/payflow/integration-guide/test-transactions/#standard-test-cards

In [4]:
# Upload csv file with test cases
import pandas as pd
df_test_cases = pd.read_csv("Pay_pal_tests.csv")
# Display the dataframe
print(df_test_cases)


                     Card_Type       Card_Number
0             American Express   378282246310005
1             American Express   371449635398431
2   American Express Corporate   378734493671000
3                  Diners Club    30569309025904
4                     Discover  6011111111111110
5                     Discover  6011000990139420
6                          JCB  3530111333300000
7                          JCB  3566002020360500
8                   Mastercard  5555555555554444
9                   Mastercard  5105105105105100
10                        Visa  4111111111111111
11                        Visa  4012888888881881
12                        Visa  4999991111111113
13                  Mastercard  5199999999999991
14                        Visa  4999992222222229
15                  Mastercard  5299999999999990
16                        Fake   123456789101112


In [5]:
# test the dfa algorithm with the test cases
for index, row in df_test_cases.iterrows():
    card_number = row['Card_Number']
    expected_result = row['Card_Type']
    result = dfa_validate(card_number)
    print(f"Testing {card_number}: Expected {expected_result}, Got {result}")

Testing 378282246310005: Expected American Express, Got AMEX
Testing 371449635398431: Expected American Express, Got AMEX
Testing 378734493671000: Expected American Express Corporate, Got AMEX
Testing 30569309025904: Expected Diners Club, Got INVALID
Testing 6011111111111110: Expected Discover, Got INVALID
Testing 6011000990139420: Expected Discover, Got INVALID
Testing 3530111333300000: Expected JCB, Got INVALID
Testing 3566002020360500: Expected JCB, Got INVALID
Testing 5555555555554444: Expected Mastercard, Got MASTERCARD
Testing 5105105105105100: Expected Mastercard, Got MASTERCARD
Testing 4111111111111111: Expected Visa, Got VISA
Testing 4012888888881881: Expected Visa, Got VISA
Testing 4999991111111113: Expected Visa, Got VISA
Testing 5199999999999991: Expected Mastercard, Got MASTERCARD
Testing 4999992222222229: Expected Visa, Got VISA
Testing 5299999999999990: Expected Mastercard, Got MASTERCARD
Testing 123456789101112: Expected Fake, Got INVALID


In [6]:
# test the standard algorithm with the test cases
for index, row in df_test_cases.iterrows():
    card_number = row['Card_Number']
    expected_result = row['Card_Type']
    result = validate_credit_card(card_number)
    print(f"Testing {card_number}: Expected {expected_result}, Got {result}")

Testing 378282246310005: Expected American Express, Got AMEX
Testing 371449635398431: Expected American Express, Got AMEX
Testing 378734493671000: Expected American Express Corporate, Got AMEX
Testing 30569309025904: Expected Diners Club, Got INVALID
Testing 6011111111111110: Expected Discover, Got INVALID
Testing 6011000990139420: Expected Discover, Got INVALID
Testing 3530111333300000: Expected JCB, Got INVALID
Testing 3566002020360500: Expected JCB, Got INVALID
Testing 5555555555554444: Expected Mastercard, Got MASTERCARD
Testing 5105105105105100: Expected Mastercard, Got MASTERCARD
Testing 4111111111111111: Expected Visa, Got VISA
Testing 4012888888881881: Expected Visa, Got VISA
Testing 4999991111111113: Expected Visa, Got VISA
Testing 5199999999999991: Expected Mastercard, Got MASTERCARD
Testing 4999992222222229: Expected Visa, Got VISA
Testing 5299999999999990: Expected Mastercard, Got MASTERCARD
Testing 123456789101112: Expected Fake, Got INVALID
